In [30]:
import json
from nltk_utils import tokenize, stem, bag_of_words
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from model import NeuralNet
from sklearn.model_selection import train_test_split

In [31]:
def callback(epoch, num_epochs, loss, acc):
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss:.4f}, Accuracy: {acc:.2f}%')


file_path = r'C:\Users\alexi\OneDrive\Desktop\Documents\BU_AI-with-Flask\intents.json' #replace with your own path, yung path ng intents.json sa files mo, right click, copy as path
with open(file_path, 'r', encoding='utf-8') as f:
    intents = json.load(f)

In [32]:
all_words = []
tags = []
xy = []

# loop through each sentence in our intents patterns
for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))

ignore_words = ['?', '!', '.', ',']

all_words = [stem(w) for w in all_words if w not in ignore_words]

all_words = sorted(set(all_words))

tags= sorted(set(tags))

In [33]:
X_train = []
y_train = []

for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)

    label = tags.index(tag)
    y_train.append(label) # CrossEntropyLoss

X_train = np.array(X_train)
y_train = np.array(y_train, dtype=np.int64)

class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train
    
    # dataset[idx]
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.n_samples

In [34]:

# Hyper-parameters
batch_size = 8
hidden_size = 8
output_size = len(tags)
input_size = len(X_train[0])
learning_rate = 0.001
num_epochs = 1000


dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [36]:

correct = 0
total = 0

for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(words)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Track the total and correct predictions
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    if (epoch+1) % 100 == 0:
        acc = 100 * correct / total
        callback(epoch, num_epochs, loss.item(), acc)

print(f'Final loss: {loss.item():.4f}, Final accuracy: {acc:.2f}%')

Epoch [100/1000], Loss: 0.5120, Accuracy: 49.75%
Epoch [200/1000], Loss: 0.1136, Accuracy: 72.74%
Epoch [300/1000], Loss: 0.0069, Accuracy: 81.83%
Epoch [400/1000], Loss: 0.0037, Accuracy: 86.37%
Epoch [500/1000], Loss: 0.0027, Accuracy: 89.10%
Epoch [600/1000], Loss: 0.0010, Accuracy: 90.91%
Epoch [700/1000], Loss: 0.0002, Accuracy: 92.21%
Epoch [800/1000], Loss: 0.0004, Accuracy: 93.19%
Epoch [900/1000], Loss: 0.0002, Accuracy: 93.94%
Epoch [1000/1000], Loss: 0.0001, Accuracy: 94.55%
Final loss: 0.0001, Final accuracy: 94.55%


In [44]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

all_labels=[]

all_predictions=[]


def plot_confusion_matrix(all_labels, all_predictions):
    cm = confusion_matrix(all_labels, all_predictions)
    sns.heatmap(cm, annot=True, fmt='g', cmap='Blues')
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.show()

# Assuming all_labels and all_predictions are defined elsewhere in your code
plot_confusion_matrix(all_labels, all_predictions)


NameError: name 'all_labels' is not defined

In [42]:
import torch
from sklearn.metrics import f1_score

def calculate_f1_score(model, data_loader, device):
    model.eval()
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    f1 = f1_score(all_labels, all_predictions, average='weighted')
    return f1

# Example usage:
# f1_score = calculate_f1_score(model, test_loader, device)
# print(f"F1 Score: {f1_score:.4f}")


F1 Score: 0.6190


In [39]:
data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "output_size": output_size,
    "hidden_size": hidden_size,
    "all_words": all_words,
    "tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'Training complete. File saved to {FILE}')

Training complete. File saved to data.pth
